In [1]:
import pandas as pd
import numpy as np
import torch
import os
import sys
from tqdm import tqdm, trange

sys.path.append("../../")
import biked_commons
from biked_commons.api import rendering

from biked_commons.resource_utils import split_datasets_path

Using java as the Java binary


This builds the overall validation function

In [2]:
from bike_bench_validation_functions import bike_bench_validation_functions
from base_validation_function import construct_tensor_validator, construct_dataframe_validator

Read in the Bike Data

In [3]:
data = pd.read_csv(split_datasets_path("bike_bench.csv"), index_col=0)
display(data)

,CS textfield,BB textfield,Stack,Head angle,Head tube length textfield,Seat stay junction0,Seat tube length,Seat angle,DT Length,FORK0R,...,Handlebar style OHCLASS: 2,Stem kind OHCLASS: 0,Stem kind OHCLASS: 1,Stem kind OHCLASS: 2,Fork type OHCLASS: 0,Fork type OHCLASS: 1,Fork type OHCLASS: 2,Seat tube type OHCLASS: 0,Seat tube type OHCLASS: 1,Seat tube type OHCLASS: 2
1,430.00,67.000000,608.32385,73.0000,135.6,45.0,588.7,72.500000,659.33480,45.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,350.00,-14.500000,476.78552,71.5000,80.0,45.0,265.0,71.500000,572.24335,45.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,415.00,4.000000,578.31660,72.0000,198.8,45.0,562.0,74.000000,647.08520,35.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,375.00,50.000000,480.25488,70.5000,91.8,40.0,460.0,73.500000,570.88070,45.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5,431.79,24.000000,504.20746,72.1000,83.1,41.0,300.0,72.500000,635.54970,45.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4796,410.00,75.000000,589.23315,72.5000,145.0,125.0,520.0,74.000000,638.39355,45.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4797,419.00,42.909344,519.15990,65.7721,95.0,90.0,380.0,77.772100,702.45337,45.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4798,410.00,-11.500000,498.85727,72.0000,250.0,45.0,550.0,74.000000,547.74940,45.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4799,410.00,-11.500000,527.76940,72.0000,280.4,45.0,550.0,74.000534,542.19660,45.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


This builds the overall validation function

In [4]:
dataframe_validator = construct_dataframe_validator(bike_bench_validation_functions)

Call the validation function and check the results. Remember, a score less or equal to 0 is valid. The more postive, the greater the magnitude of constraint violation. The more negative the further away from violating the constraint. A design is overall valid if all constraints are less or equal to 0

In [5]:
validity_scores = dataframe_validator(data)
display(validity_scores.head(10)) #design 6's saddle height is too small, for example

,Saddle height too small,Seat post too short,Head tube lower extension too great,Head tube length too great,Certain parameters must be positive,Chain stay should be greater than wheel radius,Chain stay should be greater than BB,Seat stay should be greater than wheel radius,Down tube must reach head tube,The pedal shouldn't intersect the front wheel,The crank shouldn't hit the ground when it is in its lower position,RGB value should be less than 255
1,-139.299988,-150.700012,-85.600006,-45.600006,0.0,-85.500000,-363.000000,-177.169006,-132.870605,-31.291748,-105.000000,-510.0
2,-25.000000,-65.000000,-56.900002,-33.599998,0.0,-44.500000,-364.500000,-52.448822,-149.417450,-61.552124,-147.500000,-153.0
3,-161.000000,-129.000000,-162.899994,-126.400002,0.0,-101.000000,-411.000000,-249.262878,-223.695740,-135.928223,-137.500000,-408.0
4,-100.000000,-30.000000,-57.900002,-28.800003,0.0,-63.500000,-325.000000,-122.191803,-145.204742,-39.395813,-89.000000,0.0
5,-10.000000,-230.000000,-28.799999,-8.599998,0.0,-96.790009,-407.790009,-82.747437,-158.748138,-66.727661,-138.500000,-459.0
6,5.000000,-295.000000,-65.000000,-44.900002,0.0,-68.899994,-294.899994,-86.220978,-236.341522,-95.310791,-53.500000,-408.0
7,-180.000000,-210.000000,-115.000000,-86.500000,0.0,-90.000000,-395.136963,-166.902100,-205.660156,-160.132690,-132.636978,-561.0
8,-193.000000,-197.000000,-53.599998,-26.599998,0.0,-75.500000,-382.000000,-131.034149,-201.238251,-124.511841,-134.000000,-765.0
9,-153.599976,-16.400024,-110.900002,-82.800003,0.0,-110.000000,-376.500000,-160.141876,-139.944031,-59.634583,-94.000000,-459.0
10,-333.000000,43.000000,-36.900002,-11.000000,0.0,-73.000000,-402.000000,-101.849304,-179.224426,-73.868591,-156.500000,-306.0


In [6]:
#overall valid if all constraints are less or equal to 0

validity = np.all(validity_scores <= 0, axis=1)
print(validity[:10])
print(f"Fraction of valid designs: {np.sum(validity) / len(validity)}")

1      True
2      True
3      True
4      True
5      True
6     False
7      True
8      True
9      True
10    False
dtype: bool
Fraction of valid designs: 0.9109086869584537


C:\Users\Lyle\AppData\Local\Temp\ipykernel_98688\510230081.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  print(validity[:10])


If we want to calculate gradients, we need to use the pytorch version of the validator. 

In [8]:
COLUMN_NAMES = list(data.columns)
tensor_validator, validation_names = construct_tensor_validator(bike_bench_validation_functions, COLUMN_NAMES)

In [9]:
print(validation_names)

['Saddle height too small', 'Seat post too short', 'Head tube lower extension too great', 'Head tube length too great', 'Certain parameters must be positive', 'Chain stay should be greater than wheel radius', 'Chain stay should be greater than BB', 'Seat stay should be greater than wheel radius', 'Down tube must reach head tube', "The pedal shouldn't intersect the front wheel", "The crank shouldn't hit the ground when it is in its lower position", 'RGB value should be less than 255']


In [10]:
data_tensor = torch.tensor(data.values, dtype=torch.float32, requires_grad=True)
validity_scores_tens = tensor_validator(data_tensor)
print(validity_scores_tens) # should be the same as the dataframe version

loss = validity_scores_tens[validity_scores_tens > 0].sum() # we make our objective function equal to the sum of constraint violation
loss.backward()
gradients = data_tensor.grad
gradient_df = pd.DataFrame(gradients.numpy(), columns=COLUMN_NAMES, index = data.index)

tensor([[-139.3000, -150.7000,  -85.6000,  ...,  -31.2917, -105.0000,
         -510.0000],
        [ -25.0000,  -65.0000,  -56.9000,  ...,  -61.5521, -147.5000,
         -153.0000],
        [-161.0000, -129.0000, -162.9000,  ..., -135.9282, -137.5000,
         -408.0000],
        ...,
        [-150.0000, -170.0000, -210.4000,  ..., -151.8349,  -82.0000,
         -408.0000],
        [-150.0000, -170.0000, -269.6000,  ..., -153.2349,  -82.0000,
         -408.0000],
        [-150.7000, -169.3000, -106.2000,  ...,  -51.8285, -107.5000,
         -612.0000]], grad_fn=<CopySlices>)


The gradients tell us how to adjust the parameters to ideally fix the problem

In [11]:
display(gradient_df.head(10)) # we can see that there are some gradients in the seat tube length column, for example

,CS textfield,BB textfield,Stack,Head angle,Head tube length textfield,Seat stay junction0,Seat tube length,Seat angle,DT Length,FORK0R,...,Handlebar style OHCLASS: 2,Stem kind OHCLASS: 0,Stem kind OHCLASS: 1,Stem kind OHCLASS: 2,Fork type OHCLASS: 0,Fork type OHCLASS: 1,Fork type OHCLASS: 2,Seat tube type OHCLASS: 0,Seat tube type OHCLASS: 1,Seat tube type OHCLASS: 2
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Test gradient descent to see if validity improves

In [12]:
updated_data = data
step_size = 0.1
for steps in trange(1000):
    #calculate gradients
    data_tensor = torch.tensor(updated_data.values, dtype=torch.float32, requires_grad=True)
    validity_scores_tens = tensor_validator(data_tensor)
    loss = validity_scores_tens[validity_scores_tens > 0].sum()
    loss.backward()
    gradients = data_tensor.grad
    gradients = gradients.numpy()
    updated_data = updated_data - step_size * gradients

100%|██████████| 1000/1000 [00:12<00:00, 80.56it/s]


In [13]:
#check validity of updated data
validity_scores = dataframe_validator(updated_data)
validity = np.all(validity_scores <= 0, axis=1)
print(validity.mean())

0.9940013330371029


In [14]:
display(validity_scores.head(10))


,Saddle height too small,Seat post too short,Head tube lower extension too great,Head tube length too great,Certain parameters must be positive,Chain stay should be greater than wheel radius,Chain stay should be greater than BB,Seat stay should be greater than wheel radius,Down tube must reach head tube,The pedal shouldn't intersect the front wheel,The crank shouldn't hit the ground when it is in its lower position,RGB value should be less than 255
1,-139.299988,-150.700012,-85.600006,-45.600006,0.0,-85.500000,-363.000000,-177.169006,-132.870605,-31.291748,-105.000000,-510.0
2,-25.000000,-65.000000,-56.900002,-33.599998,0.0,-44.500000,-364.500000,-52.448822,-149.417450,-61.552124,-147.500000,-153.0
3,-161.000000,-129.000000,-162.899994,-126.400002,0.0,-101.000000,-411.000000,-249.262878,-223.695740,-135.928223,-137.500000,-408.0
4,-100.000000,-30.000000,-57.900002,-28.800003,0.0,-63.500000,-325.000000,-122.191803,-145.204742,-39.395813,-89.000000,0.0
5,-10.000000,-230.000000,-28.799999,-8.599998,0.0,-96.790009,-407.790009,-82.747437,-158.748138,-66.727661,-138.500000,-459.0
6,0.000000,-290.000000,-65.000000,-44.900002,0.0,-68.899994,-294.899994,-85.047089,-236.341522,-95.310791,-53.500000,-408.0
7,-180.000000,-210.000000,-115.000000,-86.500000,0.0,-90.000000,-395.136963,-166.902100,-205.660156,-160.132690,-132.636978,-561.0
8,-193.000000,-197.000000,-53.599998,-26.599998,0.0,-75.500000,-382.000000,-131.034149,-201.238251,-124.511841,-134.000000,-765.0
9,-153.599976,-16.400024,-110.900002,-82.800003,0.0,-110.000000,-376.500000,-160.141876,-139.944031,-59.634583,-94.000000,-459.0
10,-304.199982,-0.200012,-36.900002,-11.000000,0.0,-73.000000,-402.000000,-109.148529,-179.224426,-73.868591,-156.500000,-306.0


In [15]:
margin = 0.05
parameter_ranges = np.max(data.values, axis=0) - np.min(data.values, axis=0)
lower_bounds = np.min(data.values, axis=0)-margin*parameter_ranges
upper_bounds = np.max(data.values, axis=0)+margin*parameter_ranges

#randomly sample a design
random_design = np.random.uniform(lower_bounds, upper_bounds)

#check validity of random design

validity_scores = dataframe_validator(pd.DataFrame(random_design.reshape(1, -1), columns=COLUMN_NAMES))
display(validity_scores) # should be all less than or equal to 0


,Saddle height too small,Seat post too short,Head tube lower extension too great,Head tube length too great,Certain parameters must be positive,Chain stay should be greater than wheel radius,Chain stay should be greater than BB,Seat stay should be greater than wheel radius,Down tube must reach head tube,The pedal shouldn't intersect the front wheel,The crank shouldn't hit the ground when it is in its lower position,RGB value should be less than 255
0,-430.497803,385.319031,-972.59436,-915.871277,377.272125,-219.157837,-805.832886,-160.043945,-588.518005,85.557434,-414.175079,-598.863831
